
## **Data Warehousing Analytics for Geospatial Analysis with Python and DuckDB**




## **Installing and Loading Packages**


In [1]:
# To update a package, run the following command in the terminal or command prompt:
# pip install -U package_name

# To install an exact version of a package, run the following command in the terminal or command prompt:
# !pip install package_name==desired_version

# After installing or updating the package, restart the Jupyter Notebook.

# Installs the `watermark` package.
# This package is used to log the versions of other packages used in this Notebook.
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.5 MB/s eta 0:00:00


In [2]:
%%writefile requirements.txt

duckdb
geopandas
pyarrow

Overwriting requirements.txt


In [3]:
pip install gdal


In [4]:
%pip install -r requirements.txt --quiet

In [8]:
#1. Imports

#1.a. Import the `duckdb` library for querying and managing data.
import duckdb

#1.b. Import the `geopandas` library for working with geospatial data.
import geopandas

#1.c. Import the `pyarrow` library for handling data in Apache Arrow format.
import pyarrow

In [5]:
%reload_ext watermark
%watermark -a "Your_Name"

Author: Your_Name



## **Installing DuckDB Extensions**

In [9]:
#2. Installing DuckDB extensions

#2.a. Install the `httpfs` extension for handling HTTP file systems.
duckdb.sql('INSTALL httpfs')

#2.b. Load the `httpfs` extension.
duckdb.sql('LOAD httpfs')

#2.c. Force install the `spatial` extension from the specified URL.
duckdb.sql("FORCE INSTALL spatial FROM 'http://nightly-extensions.duckdb.org';")

#2.d. Load the `spatial` extension for geospatial data processing.
duckdb.sql('LOAD spatial')

The `httpfs` extension allows us to read files directly from S3, while the `spatial` extension, as the name suggests, will be used to execute geospatial queries. Through these commands, we ensure that our DuckDB setup is well-equipped with the necessary extensions for future tasks.


## **Initializing DuckDB for In-Memory Processing**


In [10]:
#3. Initializing DuckDB for in-memory processing

#3.a. Create a DuckDB database instance with in-memory storage.
con = duckdb.connect(database=":memory:")

In [11]:
#4. Installing and loading extensions (an alternative to what was shown earlier)

#4.a. Install and load the `httpfs` extension.
con.install_extension('httpfs')
con.load_extension('httpfs')

#4.b. Install and load the `spatial` extension.
con.install_extension('spatial')
con.load_extension('spatial')

## **Loading Geospatial Data in GeoParquet Format**

See details about the data source in Chapter 11 of the course.

Now that DuckDB is configured and ready, the next step is to load spatial data of building locations. The dataset has been partitioned using two different strategies:

- **by country**
- **by country, sub-partitioned by S2 grid**

The sub-partitioning is used to optimize performance when reading GeoParquet files. By setting a limit of 20 million building areas per file, we keep the row group size somewhat optimized. Loading a large GeoParquet file directly is much slower.

Let’s start by loading a complete (large) country using a single file:


In [12]:
#5. S3 file address in the AWS cloud
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"

In [13]:
#6. Partition type
partitions = "by_country"

In [14]:
#7. Country ISO code
country_iso = "BRA"

> We created the table with data from Brazil.


In [15]:
#8. Creating a table with building data for Brazil

%%time

con.execute(f"""
    CREATE OR REPLACE TABLE brazil_buildings AS
      SELECT * FROM parquet_scan('{prefix}/by_country_s2/country_iso={country_iso}/*.parquet')
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4min 46s, sys: 1min 13s, total: 6min
Wall time: 46min 53s


## **Queries on Geospatial Data in a Table of 140 Million Records**


In [16]:
#9. Table summary

%%time

con.query('DESCRIBE brazil_buildings')

CPU times: user 2.32 ms, sys: 3.52 ms, total: 5.84 ms
Wall time: 71 ms


┌───────────────────┬────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name    │                        column_type                         │  null   │   key   │ default │  extra  │
│      varchar      │                          varchar                           │ varchar │ varchar │ varchar │ varchar │
├───────────────────┼────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ boundary_id       │ BIGINT                                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ bf_source         │ VARCHAR                                                    │ YES     │ NULL    │ NULL    │ NULL    │
│ confidence        │ DOUBLE                                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ area_in_meters    │ DOUBLE                                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ s2_id         

In [17]:
#10. Record count

%%time

con.query('SELECT COUNT(*) FROM brazil_buildings;')

CPU times: user 1.69 ms, sys: 110 µs, total: 1.8 ms
Wall time: 10.4 ms


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    141045123 │
└──────────────┘

In [18]:
#11. Selecting the source and count

%%time

con.query('SELECT bf_source AS data_source, COUNT(*) AS count FROM brazil_buildings GROUP BY data_source;')

CPU times: user 2.02 ms, sys: 152 µs, total: 2.17 ms
Wall time: 9.19 ms


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬───────────┐
│ data_source │   count   │
│   varchar   │   int64   │
├─────────────┼───────────┤
│ google      │ 136237330 │
│ microsoft   │   4807793 │
└─────────────┴───────────┘

In [19]:
#12. Using SELECT is not mandatory with DuckDB

%%time

con.query('''FROM brazil_buildings WHERE bf_source = 'google' LIMIT 10;''')

CPU times: user 2.45 ms, sys: 0 ns, total: 2.45 ms
Wall time: 11.4 ms


┌─────────────┬───────────┬────────────┬────────────────┬──────────────────────┬─────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────┬───────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ boundary_id │ bf_source │ confidence │ area_in_meters │        s2_id         │ country_iso │                                                                                               geometry                                                                                               │ geohash  │ __index_level_0__ │                                                       bbox                                                       │
│    int64    │  varchar  │   double   │     double     │        int64         │   varchar   │                          

## **Statistical Analysis by Data Partition**

Let's dive deeper into the analysis of S2 partition statistics. For this exercise, we will select a country subdivided into multiple S2 grids and calculate the building count per grid ID.

We will use Australia (**AUS**) as our country of study due to its extensive geographical distribution and substantial urban presence.


In [20]:
#13. We can work with data from other countries

prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country_s2"
country_iso = "AUS"

In [21]:
#14. Define the SQL query

query1 = f"""
    CREATE TABLE aus_buildings AS
    SELECT s2_id, COUNT(geometry) AS buildings_count
    FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')
    GROUP BY(s2_id)
"""

In [22]:
#15. Execute the query

%%time

duckdb.sql(query1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 7.93 s, sys: 1.12 s, total: 9.05 s
Wall time: 1min 9s


In the result above, Australia is divided into **three distinct S2 grids**, each with a varying number of buildings. This demonstrates the **geographical distribution** and **building density** across different regions of the country.

Now, let's calculate the **average number of buildings per S2 grid ID** to get an idea of the building density:



In [23]:
#16. Execute query

%%time

duckdb.sql("SELECT * FROM aus_buildings").show()

┌──────────────────────┬─────────────────┐
│        s2_id         │ buildings_count │
│        int64         │      int64      │
├──────────────────────┼─────────────────┤
│  3170534137668829184 │         1588372 │
│  7782220156096217088 │        10025356 │
│ -6052837899185946624 │          468519 │
└──────────────────────┴─────────────────┘

CPU times: user 2.5 ms, sys: 79 µs, total: 2.58 ms
Wall time: 3.21 ms


In [24]:
#17. Define the SQL query

query2 = f"""
    SELECT ROUND(AVG(buildings_count), 0) AS avg_num_buildings
    FROM aus_buildings
"""

In [25]:
#18. Execute the query and display the result

%%time

duckdb.sql(query2).show()

┌───────────────────┐
│ avg_num_buildings │
│      double       │
├───────────────────┤
│         4027416.0 │
└───────────────────┘

CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 11.3 ms


From the results, it is observed that, on average, there are approximately **3.7 million buildings per grid ID**. This metric provides an approximate estimate of **building density** across different geographical segments in Australia and can serve as a basis for further **spatial analyses** or comparisons with other countries.



## **Performing Geospatial Data Analysis**

Now, let's dive deeper into some analyses of our dataset, focusing on **Lesotho** (due to its smaller size). Initially, we will fetch the data from **S3** and organize it into a **DuckDB table**.


In [1]:
#19. Partition type and country ISO code

partitions = "by_country_s2"
country_iso = "LSO"

In [2]:
#20. Creating or replacing the table for Lesotho buildings

%%time

duckdb.sql(f"""
    CREATE OR REPLACE TABLE lso_buildings AS
    SELECT *
    FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')
""")

NameError: name 'duckdb' is not defined

In [ ]:
duckdb.sql(f"""
    CREATE OR REPLACE TABLE lso_buildings AS
    SELECT *, ST_AsWKB(ST_Envelope(geometry)) AS geometry
    FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
#21. Execute the query

%%time

duckdb.sql(f"""
    SELECT bf_source, COUNT(*) AS buildings_count
    FROM lso_buildings
    GROUP BY bf_source;
""").show()

Let's compare the original **Google V3 dataset** with our merged dataset to verify the absence of overlap. We will use the **Open Buildings dataset** available on Source Cooperative for this comparison.

Remember that we have already loaded **Lesotho** as a DuckDB table named `lso_buildings` in the previous steps? Now, let's do the same with the **original Google V3 building dataset**.


In [ ]:
#22. Data source from Google Research

prefix = "s3://us-west-2.opendata.source.coop/google-research-open-buildings/geoparquet-by-country"

In [ ]:
#23. Country code

country = "LS"

In [ ]:
#24. Creating or replacing the table for Lesotho buildings from Google dataset

%%time

duckdb.sql(f"""
    CREATE OR REPLACE TABLE lso_buildings_google AS
    SELECT *
    FROM '{prefix}/country_iso={country}/{country}.parquet'
""")

In [ ]:
#25. Execute the query to count records in the Google dataset for Lesotho

%%time

duckdb.sql("SELECT COUNT(*) FROM lso_buildings_google").show()

In [ ]:
#26. Execute the query to count records in Lesotho buildings filtered by Google source

%%time

duckdb.sql("SELECT COUNT(*) FROM lso_buildings WHERE bf_source = 'google'").show()

In [ ]:
#27. File with the Area of Interest (AOI)

file = "boundaries.geojson"

In [ ]:
#28. Execute the query to create a table for the Area of Interest (AOI)

%%time

duckdb.sql(f"""
    CREATE TABLE aoi AS
    SELECT *
    FROM ST_Read('{file}')
""")

In [ ]:
#29. Execute the query to display the Area of Interest (AOI) table

%%time

duckdb.sql("SELECT * FROM aoi").show()

In [ ]:
#30. Define the SQL query to clip Lesotho buildings with the Area of Interest (AOI)

query3 = """
CREATE TABLE lso_buildings_clipped AS
SELECT ST_Intersection(b.geometry, a.geom) AS geom, b.bf_source
FROM lso_buildings b, aoi a
WHERE ST_Intersects(b.geometry, a.geom);
"""

The above SQL code creates a new table called `lso_buildings_clipped` from a **geographical intersection operation** between two existing tables: `lso_buildings` and `aoi`. Let's break down each part of the code to clarify its purpose:

### **CREATE TABLE lso_buildings_clipped AS**
This SQL statement creates a new table called `lso_buildings_clipped`. The content of this table is the result of the query that follows the `AS` clause.

### **SELECT**
- `ST_Intersection(ST_GeomFromWKB(b.geometry), a.geom) AS geom`:  
  This selects the geometric intersection of the `geometry` column from the `lso_buildings` table and the `geom` column from the `aoi` table.  
  - The `ST_Intersection` function calculates the **common area** between two geometries.  
  - The `ST_GeomFromWKB` function converts the `geometry` column, stored in **Well-Known Binary (WKB)** format, into a geometry type that DuckDB can manipulate.

- `b.bf_source`:  
  This column is selected directly from the `lso_buildings` table. It contains information about the **data source** for each building.

### **FROM lso_buildings b, aoi a**
This specifies the tables from which data will be retrieved.  
- `lso_buildings` is referenced with the alias `b`.  
- `aoi` is referenced with the alias `a`.

### **WHERE ST_Intersects(ST_GeomFromWKB(b.geometry), a.geom)**
The `WHERE` clause filters records to include only those where the geometries in tables `b` and `a` **intersect**.  
- The `ST_Intersects` function returns `TRUE` if two geometries overlap in any way.

---

### **Summary**
This query creates a new table that contains:
1. The **overlapping areas** between the buildings in `lso_buildings` and the Area of Interest (AOI) in `aoi`.  
2. The `bf_source` column for each resulting geometry, indicating the **origin** of the corresponding building data.

This allows for a detailed analysis of building data within a specific area of interest, facilitating further geospatial studies or visualizations.


In [ ]:
#31. Execute the query to create the clipped table for Lesotho buildings

%%time

duckdb.sql(query3)

In [ ]:
#32. Execute the query to count records in the clipped table for Lesotho buildings

%%time

duckdb.sql("SELECT COUNT(*) FROM lso_buildings_clipped").show()

In [ ]:
#33. Define the SQL query to clip Google dataset buildings with the Area of Interest (AOI)

query4 = """
CREATE TABLE lso_buildings_google_clipped AS
SELECT ST_Intersection(b.geometry, a.geom) AS geom
FROM lso_buildings_google b, aoi a
WHERE ST_Intersects(b.geometry, a.geom);
"""

In [ ]:
#34. Execute the query to create the clipped table for Google dataset buildings

%%time

duckdb.sql(query4)

In [ ]:
#35. Execute the query to count records in the clipped table for Google dataset buildings

%%time

duckdb.sql("SELECT COUNT(*) FROM lso_buildings_google_clipped").show()

In [ ]:
#36. Define the SQL query to find buildings without intersection

query5 = """
CREATE TABLE lso_no_intersection AS
SELECT m.*
FROM lso_buildings_clipped m
WHERE NOT EXISTS (
    SELECT 1
    FROM lso_buildings_google_clipped g
    WHERE ST_Intersects(m.geom, g.geom)
);
"""

The query `query5` creates the table `lso_no_intersection`, containing all entries from the table `lso_buildings_clipped` that **do not have a geometric intersection** with any entry from the table `lso_buildings_google_clipped`.

This procedure is useful for identifying **areas or objects** that are **exclusive** to the `lso_buildings_clipped` table compared to `lso_buildings_google_clipped`.


In [ ]:
#37. Execute the query to create the table of non-intersecting buildings

%%time

duckdb.sql(query5)

In [ ]:
#38. Execute the query to count records in the non-intersecting buildings table

%%time

duckdb.sql("SELECT count(*) FROM lso_no_intersection").show()

## **Exporting the Analysis Result to Spatial Format**

After completing our analyses, it is time to export the data to the desired geospatial formats. In this case, we will use **FlatGeobuf** due to its efficiency in handling geospatial data!


In [ ]:
#39. Export the clipped Lesotho buildings dataset to FlatGeobuf format

%%time

output_file = "dataset1.fgb"
duckdb.sql(f"""
    COPY (SELECT * FROM lso_buildings_clipped)
    TO '{output_file}'
    WITH (FORMAT GDAL, DRIVER 'FlatGeobuf');
""")


In [ ]:
#40. Export the Lesotho buildings dataset with geometry transformation to FlatGeobuf format

%%time

output_file = "dataset2.fgb"
duckdb.sql(f"""
    COPY (
        SELECT *, ST_AsWKB(ST_Envelope(geometry)) AS geometry
        FROM lso_buildings
    )
    TO '{output_file}'
    WITH (FORMAT GDAL, DRIVER 'FlatGeobuf');
""")

In [ ]:
%watermark -a "Your_Name"

In [ ]:
%watermark -v -m

In [ ]:
%watermark --iversions

# **The End**